In [1]:
import os
import numpy as np
from time import time
import datetime
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from math import sqrt
import pandas as pd

filepath ='/home/dell/Xinda/SVM/server/Audio/data_vggish/vggish_features_p_add0.csv'
data = pd.read_csv(filepath)
x = data.iloc[:, :-1]  # 数据特征
y = data.iloc[:,-1]  # 标签

In [2]:

# 将数据划分为训练集和测试集，test_size=.3表示30%的测试集, 随机数种子, 保证可复现性
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=420)

# 修正测试集和训练集的索引
for i in [x_train, x_test, y_train, y_test ]:
    i.index  = range(i.shape[0])

# 标准化
scaler_x = StandardScaler()
x_train_std = scaler_x.fit_transform(x_train)
x_test_std = scaler_x.fit_transform(x_test)
print(x_train_std.shape)
print(y_train.shape)



def getPvar(vals, mean):
    N = len(vals)
    su = 0
    for i in range(len(vals)):
        su = su + ((vals[i]-mean)*(vals[i]-mean))
    pvar = (1/N) * su
    return pvar

def getMean(vals):
    su = 0
    for i in range(len(vals)):
        su = su + vals[i]
    mean = su/(len(vals))
    return mean

def getMeanofDiffs(xvals, yvals):
    su = 0
    for i in range(len(xvals)):
        su = su + ((xvals[i] - yvals[i])*(xvals[i] - yvals[i]))
    meanodiffs = su/(len(xvals))
    return meanodiffs

def getCCC(pvarfe,pvarexp,meanofdiff,meanfe,meanexp):
    bottom = pvarfe + pvarexp + ((meanfe - meanexp)*(meanfe - meanexp))
    answer = 1 - (meanofdiff / bottom)
    return answer


(10244, 384)
(10244,)


In [4]:
y_test

0       0.00
1       0.00
2       0.75
3       0.00
4       1.00
        ... 
4386    0.00
4387    0.50
4388    0.00
4389    0.00
4390   -0.25
Name: Pleasure_average, Length: 4391, dtype: float64

## Gamma

In [ ]:
times_all = time()
# 调试两个参数 gamma & C  ，默认情况下C为1，通常来说这都是一个合理的参数。
RMSE_list = []
Spearman_list = []
CCC_list = []

# =========================================== Gamma
gamma_range = np.logspace(-10, 1, 10, base=2)  # 返回13个数字，底是2
print("gamma_rang:", gamma_range)

for gamma_item in gamma_range:
    count = 1
    time0 = time()
    print("Start-{0}, gamma={1}".format(count, gamma_item))
    count = count + 1
    clf = SVR(kernel='rbf', gamma=gamma_item, cache_size=5000)
    clf.fit(x_train_std, y_train)

    result = clf.predict(x_test_std)
    # accuracy
    rmse = sqrt(mean_squared_error(y_test, result))
    RMSE_list.append(rmse)
    print("(1) Evaluation - RMSE = ", rmse)

    # Spearman
    data = {'result': result, 'y_test': y_test}
    df = pd.DataFrame(data, columns=['result', 'y_test'])
    spearman = df.corr(method="spearman")
    print("(2) Evaluation - Spearmman = \n", spearman)
    Spearman_list.append(spearman)

    # CCC
    prediction = result
    ground = y_test
    meanfe = getMean(ground)
    meanexp = getMean(prediction)
    meanofdiff = getMeanofDiffs(ground, prediction)
    pvarfe = getPvar(ground, meanfe)
    pvarexp = getPvar(prediction, meanexp)
    ccc = getCCC(pvarfe, pvarexp, meanofdiff, meanfe, meanexp)
    CCC_list.append(ccc)
    print('(3) Evaluation - CCC =  ' + str(ccc))
    print(datetime.datetime.fromtimestamp(time() - time0).strftime("%M:%S:%f"))
    print()
    print()

print("Gamma = ", gamma_range[RMSE_list.index(min(RMSE_list))])
print("RMSE = ", min(RMSE_list))
print("Spearman = ", Spearman_list[RMSE_list.index(min(RMSE_list))])
print("CCC = ", CCC_list[RMSE_list.index(min(RMSE_list))])
print(datetime.datetime.fromtimestamp(time() - times_all).strftime("%M:%S:%f"))

gamma_rang: [9.76562500e-04 2.27837703e-03 5.31558594e-03 1.24015707e-02
 2.89335848e-02 6.75037337e-02 1.57490131e-01 3.67433623e-01
 8.57243983e-01 2.00000000e+00]
Start-1, gamma=0.0009765625
(1) Evaluation - RMSE =  0.334731135136267
(2) Evaluation - Spearmman = 
           result    y_test
result  1.000000  0.146721
y_test  0.146721  1.000000
(3) Evaluation - CCC =  0.05796762242305764
02:57:171967


Start-1, gamma=0.0022783770304221013
(1) Evaluation - RMSE =  0.3364055099987637
(2) Evaluation - Spearmman = 
           result    y_test
result  1.000000  0.148783
y_test  0.148783  1.000000
(3) Evaluation - CCC =  0.08465675885825441
03:21:370746


Start-1, gamma=0.005315585938181161
(1) Evaluation - RMSE =  0.33777596324307363
(2) Evaluation - Spearmman = 
           result    y_test
result  1.000000  0.139533
y_test  0.139533  1.000000
(3) Evaluation - CCC =  0.09205101644959723
03:50:401669


Start-1, gamma=0.012401570718501561
(1) Evaluation - RMSE =  0.33602040966591634
(2) Eva

## C

In [3]:
times_all = time()
# 调试两个参数 gamma & C  ，默认情况下C为1，通常来说这都是一个合理的参数。
RMSE_list = []
Spearman_list = []
CCC_list = []

# =========================================== Gamma
C_range = np.logspace(-10, 1, 10, base=2)  # 返回13个数字，底是2
print("gamma_rang:", C_range)

for C_item in C_range:
    count = 1
    time0 = time()
    print("Start-{0}, c={1}".format(count, C_item))
    count = count + 1
    clf = SVR(kernel='rbf', gamma=0.005315585938181161,C=C_item , cache_size=5000)
    clf.fit(x_train_std, y_train)

    result = clf.predict(x_test_std)
    # accuracy
    rmse = sqrt(mean_squared_error(y_test, result))
    RMSE_list.append(rmse)
    print("(1) Evaluation - RMSE = ", rmse)

    # Spearman
    data = {'result': result, 'y_test': y_test}
    df = pd.DataFrame(data, columns=['result', 'y_test'])
    spearman = df.corr(method="spearman")
    print("(2) Evaluation - Spearmman = \n", spearman)
    Spearman_list.append(spearman)

    # CCC
    prediction = result
    ground = y_test
    meanfe = getMean(ground)
    meanexp = getMean(prediction)
    meanofdiff = getMeanofDiffs(ground, prediction)
    pvarfe = getPvar(ground, meanfe)
    pvarexp = getPvar(prediction, meanexp)
    ccc = getCCC(pvarfe, pvarexp, meanofdiff, meanfe, meanexp)
    CCC_list.append(ccc)
    print('(3) Evaluation - CCC =  ' + str(ccc))
    print(datetime.datetime.fromtimestamp(time() - time0).strftime("%M:%S:%f"))
    print()
    print()

print("C = ", max(CCC_list))
print("RMSE = ",RMSE_list[CCC_list.index(max(CCC_list))])
print("Spearman = ", Spearman_list[CCC_list.index(max(CCC_list))])
print("CCC = ", max(CCC_list))
print(datetime.datetime.fromtimestamp(time() - times_all).strftime("%M:%S:%f"))

#%%

gamma_rang: [9.76562500e-04 2.27837703e-03 5.31558594e-03 1.24015707e-02
 2.89335848e-02 6.75037337e-02 1.57490131e-01 3.67433623e-01
 8.57243983e-01 2.00000000e+00]
Start-1, c=0.0009765625
(1) Evaluation - RMSE =  0.33294131439525165
(2) Evaluation - Spearmman = 
           result    y_test
result  1.000000  0.158821
y_test  0.158821  1.000000
(3) Evaluation - CCC =  0.000413917215551729
02:45:119207


Start-1, c=0.0022783770304221013
(1) Evaluation - RMSE =  0.33285067765597354
(2) Evaluation - Spearmman = 
           result    y_test
result  1.000000  0.168527
y_test  0.168527  1.000000
(3) Evaluation - CCC =  0.0009732604386464283
03:00:015507


Start-1, c=0.005315585938181161
(1) Evaluation - RMSE =  0.332634604745564
(2) Evaluation - Spearmman = 
           result    y_test
result  1.000000  0.172057
y_test  0.172057  1.000000
(3) Evaluation - CCC =  0.0023740486345207312
03:16:211622


Start-1, c=0.012401570718501561
(1) Evaluation - RMSE =  0.33216175199072967
(2) Evaluation - 

## Prediciton

In [3]:
RMSE_list = []
Spearman_list = []
CCC_list = []
clf = SVR(kernel = 'rbf', gamma = 0.005315585938181161, C=2, cache_size=5000)
clf.fit(x_train_std, y_train)
result = clf.predict(x_test_std)
# accuracy
rmse = sqrt(mean_squared_error(y_test, result))
print("(1) Evaluation - RMSE = ", rmse)

# Spearman
data = {'result':result, 'y_test':y_test}
df = pd.DataFrame(data, columns=['result','y_test'])
spearman = df.corr(method="spearman" )
print("(2) Evaluation - Spearmman = \n", spearman)

# CCC
prediction = result
ground = y_test
meanfe = getMean(ground)
meanexp = getMean(prediction)
meanofdiff = getMeanofDiffs(ground,prediction)
pvarfe = getPvar(ground, meanfe)
pvarexp = getPvar(prediction, meanexp)
ccc = getCCC(pvarfe,pvarexp,meanofdiff,meanfe,meanexp)
print('(3) Evaluation - CCC =  ' + str(ccc))

df = pd.DataFrame(data={"vggish_prediction_p": prediction, "vggish_groundtruth_p": y_test.values.tolist()})
df.to_csv("eval_vggish_pleasure.csv")
print("save success!")

(1) Evaluation - RMSE =  0.34317130984703864
(2) Evaluation - Spearmman = 
           result    y_test
result  1.000000  0.133692
y_test  0.133692  1.000000
(3) Evaluation - CCC =  0.10045343514251548
save success!


In [4]:
df.to_csv("eval_vggish_pleasure_add0_420.csv")
print("save success!")

save success!
